In [1]:
import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import collections

import time
from filterpy.kalman import KalmanFilter

In [2]:
no_of_future_frames = 5
no_of_past_frames = 5

In [3]:
def linear_assignment(cost_matrix):
    try:
        import lap
        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i],i] for i in x if i >= 0]) #
    except ImportError:
        from scipy.optimize import linear_sum_assignment
        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

In [4]:
def iou_batch(bb_test, bb_gt):
  """
  From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
  """
  bb_gt = np.expand_dims(bb_gt, 0)
  bb_test = np.expand_dims(bb_test, 1)
  
  xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
  yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
  xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
  yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
  return(o)  


def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))

def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))

In [23]:
class KalmanBoxTracker(object):
    """
    This class represents the internal state of individual tracked objects observed as bbox.
    """
    count = 0
    def __init__(self,bbox,no_of_past_frames=5,no_of_future_frames=5):
        """
        Initialises a tracker using initial bounding box.
        """
        #define constant velocity model
        self.kf = KalmanFilter(dim_x=7, dim_z=4) 
        self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
        self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

        self.kf.R[2:,2:] *= 10.
        self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01

        self.kf.x[:4] = convert_bbox_to_z(bbox)
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []

        self.no_of_past_frames = no_of_past_frames
        self.no_of_future_frames = no_of_future_frames
        self.update_history = collections.deque(maxlen=self.no_of_past_frames)
        self.predict_history = collections.deque(maxlen=self.no_of_future_frames)

        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self,bbox):
        """
        Updates the state vector with observed bbox.
        """
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        self.kf.update(convert_bbox_to_z(bbox))
        self.update_history.append(bbox)

        if  len(self.update_history)>=self.no_of_past_frames :
            kbt = KalmanBoxTrajectory(self.update_history[0])
            for j in range(1,self.no_of_past_frames):
                kbt.predict()
                p = kbt.update(self.updateself._history[j])
            for k in range(self.no_of_future_frames):
                p = kbt.predict()[0]
                self.predict_history.append(p)


    def predict(self):
        """
        Advances the state vector and returns the predicted bounding box estimate.
        """
        if((self.kf.x[6]+self.kf.x[2])<=0):
          self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age += 1
        if(self.time_since_update>0):
          self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(convert_x_to_bbox(self.kf.x))
        return self.history[-1]

    def get_state(self):
        """
        Returns the current bounding box estimate.
        """
        return convert_x_to_bbox(self.kf.x)

    def get_future_predictions(self):
        """
        Returns the future bounding box estimate.
        """
        return np.array(self.predict_history)

In [6]:
class KalmanBoxTrajectory(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.id = KalmanBoxTrajectory.count
    KalmanBoxTrajectory.count += 1
    self.history = []

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.history = []
    self.kf.update(convert_bbox_to_z(bbox))    

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)

In [7]:
def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)

  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)

  iou_matrix = iou_batch(detections, trackers)

  if min(iou_matrix.shape) > 0:
    a = (iou_matrix > iou_threshold).astype(np.int32)
    if a.sum(1).max() == 1 and a.sum(0).max() == 1:
        matched_indices = np.stack(np.where(a), axis=1)
    else:
      matched_indices = linear_assignment(-iou_matrix)
  else:
    matched_indices = np.empty(shape=(0,2))

  unmatched_detections = []
  for d, det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t, trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0], m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)

### test functions

In [7]:
l = collections.deque(maxlen=no_of_past_frames)
print(len(l))

0


In [8]:
# predict and update simultaniously
f = KalmanBoxTracker([0,1,2,3])
# fc = KalmanBoxTracker(f.history[-1][0])
true=[]
predicted=[]

for i in range(5):
    z = [i+1, i+2, i+3, i+4]
    true.append(z)
    p = f.predict()[0]
    f.update(z)
    predicted.append(p[:2])
    print('len history ', len(f.history))
    print('predicted :\n', p)
    print('Measured : ', z)
    plt.plot(z[0],z[1], c='blue', marker='o', markersize=6)
    plt.plot(p[0],p[1], c='red', marker='*', markersize=3)
# plt.show()

len history  0
predicted :
 [0. 1. 2. 3.]
Measured :  [1, 2, 3, 4]
len history  0
predicted :
 [1.99870156 2.99870156 3.99870156 4.99870156]
Measured :  [2, 3, 4, 5]
len history  0
predicted :
 [2.99971793 3.99971793 4.99971793 5.99971793]
Measured :  [3, 4, 5, 6]
len history  0
predicted :
 [3.99987343 4.99987343 5.99987343 6.99987343]
Measured :  [4, 5, 6, 7]
len history  0
predicted :
 [4.99992445 5.99992445 6.99992445 7.99992445]
Measured :  [5, 6, 7, 8]


In [9]:
# predict 5 and update 5
f = KalmanBoxTracker([0,1,2,3])
# fc = KalmanBoxTracker(f.history[-1][0])
true=[]
predicted_All=[]
predicted = collections.deque(maxlen=no_of_past_frames)
future_bb = []

for i in range(1,16):
    
    z = [i, i+1, i+2, i+3]
    true.append(z)
    p = f.predict()[0]
    f.update(z)
    predicted.append(p[:2])
    predicted_All.append(p[:2])
    print('\nlen update_history ', len(f.update_history))
    print('hits counter ', f.hits, ' hit_streak ',f.hit_streak,' age ',f.age)
    print('predicted :\n', p)
    print('Measured : ', z)
    plt.plot(z[0],z[1], c='yellow', marker='o', markersize=12)
    plt.plot(p[0],p[1], c='red', marker='*', markersize=9)
    
    if  i%no_of_past_frames == 0 and i>1:
        print('\n--------- %s predicted future_frames: ---------\n'%i)
        fc = KalmanBoxTrajectory(f.update_history[0])
        
        for j in range(1,no_of_past_frames):
            fc.predict()
            p = fc.update(f.update_history[j])
            
        for k in range(no_of_future_frames):
            p = fc.predict()[0]
            f.predict_history.append(p)
            plt.plot(p[0],p[1], c='green', marker='o', markersize=18)
            print('predicted future_frames:\n', p)
        print('\n---------  predict_history: ---------\n',f.predict_history)
        print('\n--------- %s end of predicted future_frames: ---------\n'%i)
            
    
plt.show()


len update_history  1
hits counter  1  hit_streak  1  age  1
predicted :
 [0. 1. 2. 3.]
Measured :  [1, 2, 3, 4]

len update_history  2
hits counter  2  hit_streak  2  age  2
predicted :
 [1.99870156 2.99870156 3.99870156 4.99870156]
Measured :  [2, 3, 4, 5]

len update_history  3
hits counter  3  hit_streak  3  age  3
predicted :
 [2.99971793 3.99971793 4.99971793 5.99971793]
Measured :  [3, 4, 5, 6]

len update_history  4
hits counter  4  hit_streak  4  age  4
predicted :
 [3.99987343 4.99987343 5.99987343 6.99987343]
Measured :  [4, 5, 6, 7]

len update_history  5
hits counter  5  hit_streak  5  age  5
predicted :
 [4.99992445 5.99992445 6.99992445 7.99992445]
Measured :  [5, 6, 7, 8]

--------- 5 predicted future_frames: ---------

predicted future_frames:
 [5.99992445 6.99992445 7.99992445 8.99992445]
predicted future_frames:
 [6.99987663 7.99987663 8.99987663 9.99987663]
predicted future_frames:
 [ 7.99982881  8.99982881  9.99982881 10.99982881]
predicted future_frames:
 [ 8.999

In [10]:
bbox = [1,2,3,4]
f=KalmanBoxTracker(bbox)

for i in range(2,10):
    print(i)
    f.update([i,i+1,i+2,i+3])
    print(f.get_state(),'\n')
    print(f.get_future_predictions(),end='\n\n')

2
[[1.90909091 2.90909091 3.90909091 4.90909091]] 

[]

3
[[2.42857143 3.42857143 4.42857143 5.42857143]] 

[]

4
[[2.93548387 3.93548387 4.93548387 5.93548387]] 

[]

5
[[3.43902439 4.43902439 5.43902439 6.43902439]] 

[]

6
[[3.94117647 4.94117647 5.94117647 6.94117647]] 

[[ 6.99992445  7.99992445  8.99992445  9.99992445]
 [ 7.99987663  8.99987663  9.99987663 10.99987663]
 [ 8.99982881  9.99982881 10.99982881 11.99982881]
 [ 9.99978098 10.99978098 11.99978098 12.99978098]
 [10.99973316 11.99973316 12.99973316 13.99973316]]

7
[[4.44262295 5.44262295 6.44262295 7.44262295]] 

[[ 7.99992445  8.99992445  9.99992445 10.99992445]
 [ 8.99987663  9.99987663 10.99987663 11.99987663]
 [ 9.99982881 10.99982881 11.99982881 12.99982881]
 [10.99978098 11.99978098 12.99978098 13.99978098]
 [11.99973316 12.99973316 13.99973316 14.99973316]]

8
[[4.94366197 5.94366197 6.94366197 7.94366197]] 

[[ 8.99992445  9.99992445 10.99992445 11.99992445]
 [ 9.99987663 10.99987663 11.99987663 12.99987663]
 [10

In [11]:
print(f.id)
d=f.get_state()[0]
print(d)
fp=f.get_future_predictions()
print(fp)

2
[5.44444444 6.44444444 7.44444444 8.44444444]
[[ 9.99992445 10.99992445 11.99992445 12.99992445]
 [10.99987663 11.99987663 12.99987663 13.99987663]
 [11.99982881 12.99982881 13.99982881 14.99982881]
 [12.99978098 13.99978098 14.99978098 15.99978098]
 [13.99973316 14.99973316 15.99973316 16.99973316]]


In [12]:
# ret=np.array([])
# traj_output = np.array([])
# traj_output = np.append(traj_output, (d,fp,f.id))
# print(traj_output.shape)
# ret = np.hstack((ret,traj_output))
ret=[]
traj_output = []
traj_output.append(d)
traj_output.append(fp)
traj_output.append(f.id)
ret.append(traj_output)
print(np.array(traj_output).shape)
print(traj_output)
print(np.array(ret).shape)
print(ret)

(3,)
[array([5.44444444, 6.44444444, 7.44444444, 8.44444444]), array([[ 9.99992445, 10.99992445, 11.99992445, 12.99992445],
       [10.99987663, 11.99987663, 12.99987663, 13.99987663],
       [11.99982881, 12.99982881, 13.99982881, 14.99982881],
       [12.99978098, 13.99978098, 14.99978098, 15.99978098],
       [13.99973316, 14.99973316, 15.99973316, 16.99973316]]), 2]
(1, 3)
[[array([5.44444444, 6.44444444, 7.44444444, 8.44444444]), array([[ 9.99992445, 10.99992445, 11.99992445, 12.99992445],
       [10.99987663, 11.99987663, 12.99987663, 13.99987663],
       [11.99982881, 12.99982881, 13.99982881, 14.99982881],
       [12.99978098, 13.99978098, 14.99978098, 15.99978098],
       [13.99973316, 14.99973316, 15.99973316, 16.99973316]]), 2]]


/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if sys.path[0] == '':
/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [13]:
traj_output = []
traj_output.append(d)
traj_output.append(fp)
traj_output.append(f.id)
ret.append(traj_output)
print(np.array(traj_output).shape)
print(traj_output)
print(np.array(ret).shape)
print(ret)

(3,)
[array([5.44444444, 6.44444444, 7.44444444, 8.44444444]), array([[ 9.99992445, 10.99992445, 11.99992445, 12.99992445],
       [10.99987663, 11.99987663, 12.99987663, 13.99987663],
       [11.99982881, 12.99982881, 13.99982881, 14.99982881],
       [12.99978098, 13.99978098, 14.99978098, 15.99978098],
       [13.99973316, 14.99973316, 15.99973316, 16.99973316]]), 2]
(2, 3)
[[array([5.44444444, 6.44444444, 7.44444444, 8.44444444]), array([[ 9.99992445, 10.99992445, 11.99992445, 12.99992445],
       [10.99987663, 11.99987663, 12.99987663, 13.99987663],
       [11.99982881, 12.99982881, 13.99982881, 14.99982881],
       [12.99978098, 13.99978098, 14.99978098, 15.99978098],
       [13.99973316, 14.99973316, 15.99973316, 16.99973316]]), 2], [array([5.44444444, 6.44444444, 7.44444444, 8.44444444]), array([[ 9.99992445, 10.99992445, 11.99992445, 12.99992445],
       [10.99987663, 11.99987663, 12.99987663, 13.99987663],
       [11.99982881, 12.99982881, 13.99982881, 14.99982881],
       [1

/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [14]:
ret=[]
#1
d = f.get_state()[0] #current bounding box position
fp=f.get_future_predictions() #future predictions
ret.append([d,fp, f.id]) 
#2
d = f.get_state()[0] #current bounding box position
fp=f.get_future_predictions() #future predictions
ret.append([d,fp, f.id]) 

print(np.array(ret).shape)
print(ret)

(2, 3)
[[array([5.44444444, 6.44444444, 7.44444444, 8.44444444]), array([[ 9.99992445, 10.99992445, 11.99992445, 12.99992445],
       [10.99987663, 11.99987663, 12.99987663, 13.99987663],
       [11.99982881, 12.99982881, 13.99982881, 14.99982881],
       [12.99978098, 13.99978098, 14.99978098, 15.99978098],
       [13.99973316, 14.99973316, 15.99973316, 16.99973316]]), 2], [array([5.44444444, 6.44444444, 7.44444444, 8.44444444]), array([[ 9.99992445, 10.99992445, 11.99992445, 12.99992445],
       [10.99987663, 11.99987663, 12.99987663, 13.99987663],
       [11.99982881, 12.99982881, 13.99982881, 14.99982881],
       [12.99978098, 13.99978098, 14.99978098, 15.99978098],
       [13.99973316, 14.99973316, 15.99973316, 16.99973316]]), 2]]


/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [15]:
# predict future positions if past position limit {no_of_past_frames} is reached
f = KalmanBoxTracker([0,1,2,3])
# fc = KalmanBoxTracker(f.history[-1][0])
true=[]
predicted_All=[]
predicted = collections.deque(maxlen=no_of_past_frames)
future_bb = []

for i in range(1,16):
    
    z = [i, i+1, i+2, i+3]
    true.append(z)
    p = f.predict()[0]
    f.update(z)
    predicted.append(p[:2])
    predicted_All.append(p[:2])
    print('\nlen update_history ', len(f.update_history))
    print('hits counter ', f.hits, ' hit_streak ',f.hit_streak,' age ',f.age)
    print('predicted :\n', p)
    print('Measured : ', z)
    plt.plot(z[0],z[1], c='yellow', marker='o', markersize=12)
    plt.plot(p[0],p[1], c='red', marker='*', markersize=9)
    
    if  len(f.update_history)>=no_of_past_frames :
        print('\n--------- %s predicted future_frames: ---------\n'%i)
        fc = KalmanBoxTrajectory(f.update_history[0])
        
        for j in range(1,no_of_past_frames):
            fc.predict()
            p = fc.update(f.update_history[j])
            
        for k in range(no_of_future_frames):
            p = fc.predict()[0]
            f.predict_history.append(p)
            plt.plot(p[0],p[1], c='green', marker='o', markersize=18)
            print('predicted future_frames:\n', p)
        print('\n---------  predict_history: ---------\n',f.predict_history)
        print('\n--------- %s end of predicted future_frames: ---------\n'%i)
            
    
plt.show()


len update_history  1
hits counter  1  hit_streak  1  age  1
predicted :
 [0. 1. 2. 3.]
Measured :  [1, 2, 3, 4]

len update_history  2
hits counter  2  hit_streak  2  age  2
predicted :
 [1.99870156 2.99870156 3.99870156 4.99870156]
Measured :  [2, 3, 4, 5]

len update_history  3
hits counter  3  hit_streak  3  age  3
predicted :
 [2.99971793 3.99971793 4.99971793 5.99971793]
Measured :  [3, 4, 5, 6]

len update_history  4
hits counter  4  hit_streak  4  age  4
predicted :
 [3.99987343 4.99987343 5.99987343 6.99987343]
Measured :  [4, 5, 6, 7]

len update_history  5
hits counter  5  hit_streak  5  age  5
predicted :
 [4.99992445 5.99992445 6.99992445 7.99992445]
Measured :  [5, 6, 7, 8]

--------- 5 predicted future_frames: ---------

predicted future_frames:
 [5.99992445 6.99992445 7.99992445 8.99992445]
predicted future_frames:
 [6.99987663 7.99987663 8.99987663 9.99987663]
predicted future_frames:
 [ 7.99982881  8.99982881  9.99982881 10.99982881]
predicted future_frames:
 [ 8.999

### Trajectory class

In [27]:
#trajectory class
class Trajectory(object):
  def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3, no_of_future_frames=5, no_of_past_frames=5):
    """
    Sets key parameters for Trajectory
    """
    self.max_age = max_age
    self.min_hits = min_hits
    self.iou_threshold = iou_threshold
    self.trackers = []
    self.frame_count = 0

    self.no_of_future_frames = no_of_future_frames
    self.no_of_past_frames = no_of_past_frames


  def update(self, dets=np.empty((0, 5))):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
    Returns the a similar array, where the last column is the object ID.

    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    # get predicted locations from existing trackers.
    trks = np.zeros((len(self.trackers), 5))
    to_del = []
    ret = []
    for t, trk in enumerate(trks):
      pos = self.trackers[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
      if np.any(np.isnan(pos)):
        to_del.append(t)
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks, self.iou_threshold)

    # update matched trackers with assigned detections
    for m in matched:
      self.trackers[m[1]].update(dets[m[0], :])

    # create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        trk = KalmanBoxTracker(dets[i,:],self.no_of_past_frames,self.no_of_future_frames)
        self.trackers.append(trk)
    i = len(self.trackers)
    for trk in reversed(self.trackers):
        d = trk.get_state()[0] #current bounding box position
        fp = trk.get_future_predictions() #future predictions
        if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
          ret.append([d,fp, trk.id]) 
        i -= 1
        # remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers.pop(i)
    if(len(ret)>0):
      return np.array(ret)
    return np.empty((0,5))

In [21]:
#create instance of Trajectory
mot_tracker = Trajectory() 

In [24]:
 """
            Params:
              dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
            Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
            Returns the a similar array, where the last column is the object ID.

            NOTE: The number of objects returned may differ from the number of detections provided.
"""
detectionsNumPyArray=np.array([[0,1,2,3,0.9851834],])

In [25]:
# track_bbs_ids is a np array where each row contains a present and future bounding boxes and track_id (last column)
track_bbs_ids = mot_tracker.update(detectionsNumPyArray)
track_bbs_ids

/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/ipykernel_launcher.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


array([[array([0., 1., 2., 3.]), array([], dtype=float64), 0]],
      dtype=object)

In [26]:
for i in range(1,10):
    print(i)
    detectionsNumPyArray=np.array([[i,i+1,i+2,i+3,0.9851834],])
    track_bbs_ids = mot_tracker.update(detectionsNumPyArray)
    print(track_bbs_ids)

1
[[array([1., 2., 3., 4.]) array([], dtype=float64) 1]]
2
[[array([2., 3., 4., 5.]) array([], dtype=float64) 2]]
3
[]
4
[]
5
[]
6
[]
7
[]
8
[]
9
[]


/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/ipykernel_launcher.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
